In [2]:
# from trade_ibkr.app import run_ib_server
# from trade_ibkr.model import OnExecutionFetchedEvent
#
# def store_executions():
#     app = run_ib_server()
#
#     async def fetched(e: OnExecutionFetchedEvent):
#         for identifier, df in e.executions.execution_dataframes.items():
#             dest = f"execution-{identifier}.csv"
#             print(f"Saved to: {dest}")
#             df.to_csv(dest)
#
#         app.disconnect()
#
#     app.set_on_executions_fetched(
#         on_execution_fetched=fetched,
#         on_execution_fetched_params=app._execution_on_fetched_params
#     )
#     app.request_all_executions()
#
# store_executions()

In [3]:
import pandas as pd

def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return df.groupby("date").sum()[["realized_pnl", "px_side"]]

get_executions_df("../../execution-477837024.csv")

,realized_pnl,px_side
date,,
2022-03-18,2326.020001,320.211667
2022-03-21,-1456.600001,-306.435834
2022-03-22,943.200000,117.325000
2022-03-23,-458.300000,-80.706667


In [4]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    return df, df.groupby("date").sum()[cols], df.groupby("date").count()[cols]

df_all, df_sum, df_count = get_executions_ungroup("../../execution-477837024.csv")
df_all

,Unnamed: 0,contract,time_completed,side,quantity,avg_price,realized_pnl,epoch_sec,profit,loss,...,avg_pnl_loss,avg_pnl_rr,avg_pnl_ewr,total_px_profit,avg_px_profit,total_px_loss,avg_px_loss,avg_px_rr,avg_px_ewr,date
2,2,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 01:09:43.000,SLD,2,14026.50,155.720000,1.647584e+09,1,0,...,NaN,NaN,NaN,38.930000,38.930000,0.000000,NaN,NaN,NaN,2022-03-18
5,5,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 02:03:43.000,SLD,1,14014.25,-41.306667,1.647587e+09,1,1,...,-41.306667,3.769851,0.209650,0.000000,38.930000,-20.653333,-20.653333,1.884926,0.346629,2022-03-18
6,6,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 02:51:33.000,SLD,2,14037.50,10.386667,1.647590e+09,2,1,...,-41.306667,2.010652,0.332154,41.526667,20.763333,0.000000,-20.653333,1.005326,0.498672,2022-03-18
8,8,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 05:09:56.000,SLD,3,14034.50,11.580000,1.647598e+09,3,1,...,-41.306667,1.433882,0.410866,43.456667,14.485556,0.000000,-20.653333,0.701366,0.587763,2022-03-18
10,10,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 06:08:28.000,SLD,3,13990.00,-82.920000,1.647602e+09,3,2,...,-62.113334,0.953562,0.511886,0.000000,14.485556,-34.473333,-17.236667,0.840392,0.543363,2022-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-23 11:46:03.000,SLD,5,14573.75,-411.200000,1.648054e+09,31,28,...,-249.652381,1.221813,0.450083,0.000000,39.207285,-1053.901667,-37.639345,1.041657,0.489798,2022-03-23
206,206,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-23 12:17:19.000,SLD,5,14555.75,-349.700000,1.648056e+09,31,29,...,-253.102299,1.205159,0.453482,0.000000,39.207285,-1088.871667,-37.547299,1.044211,0.489186,2022-03-23
210,210,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-23 13:10:36.000,SLD,5,14472.00,-451.700000,1.648059e+09,31,30,...,-259.722222,1.174442,0.459888,0.000000,39.207285,-1134.041667,-37.801389,1.037192,0.490872,2022-03-23
214,214,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-23 13:57:18.000,SLD,5,14503.25,111.300000,1.648062e+09,32,30,...,-259.722222,1.151132,0.464872,1226.555834,38.329870,0.000000,-37.801389,1.013980,0.496529,2022-03-23


In [5]:
df_sum


,quantity,realized_pnl,px_side
date,,,
2022-03-18,57,2326.020001,320.211667
2022-03-21,90,-1456.600001,-306.435834
2022-03-22,20,943.200000,117.325000
2022-03-23,45,-458.300000,-80.706667


In [6]:
df_count


,quantity,realized_pnl,px_side
date,,,
2022-03-18,19,19,19
2022-03-21,27,27,27
2022-03-22,5,5,5
2022-03-23,12,12,12


In [7]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]], df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]

df_pos, df_neg = get_executions_ungroup("../../execution-477837024.csv")

In [8]:
df_pos


,realized_pnl
date,
2022-03-18,2943.506668
2022-03-21,3122.900000
2022-03-22,1809.460000
2022-03-23,1691.320000


In [9]:
df_neg

,realized_pnl
date,
2022-03-18,-617.486667
2022-03-21,-4579.500001
2022-03-22,-866.260000
2022-03-23,-2149.620000


e:\personal@ssd\programming projects\other\rx-trade-ib\venv-310\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,Unnamed: 0,contract,time_completed,side,quantity,avg_price,realized_pnl,epoch_sec,profit,loss,...,avg_pnl_loss,avg_pnl_rr,avg_pnl_ewr,total_px_profit,avg_px_profit,total_px_loss,avg_px_loss,avg_px_rr,avg_px_ewr,date
66,66,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 06:11:16.000,SLD,3,14387.25,200.580000,1.647861e+09,15,5,...,-123.497333,1.697249,0.370748,464.750000,30.983333,0.000000,-22.221667,1.394285,0.417661,2022-03-21
71,71,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 06:41:59.000,SLD,4,14406.00,-79.560000,1.647863e+09,15,6,...,-116.174444,1.804233,0.356604,0.000000,30.983333,-121.053333,-20.175556,1.535687,0.394370,2022-03-21
75,75,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 06:58:21.000,SLD,4,14397.75,-92.560000,1.647864e+09,15,7,...,-112.800952,1.858192,0.349872,0.000000,30.983333,-132.623333,-18.946191,1.635333,0.379459,2022-03-21
79,79,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 07:27:40.000,SLD,4,14378.75,-142.560000,1.647866e+09,15,8,...,-116.520833,1.798870,0.357287,0.000000,30.983333,-150.443333,-18.805417,1.647575,0.377704,2022-03-21
82,82,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 08:38:09.000,SLD,4,14421.00,386.440000,1.647870e+09,16,8,...,-116.520833,1.893721,0.345576,513.055000,32.065938,0.000000,-18.805417,1.705144,0.369666,2022-03-21
88,88,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 08:55:58.999,SLD,5,14304.50,-1293.200000,1.647871e+09,16,9,...,-247.262963,0.892402,0.528429,0.000000,32.065938,-279.763333,-31.084815,1.031563,0.492232,2022-03-21
90,90,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 08:57:57.000,BOT,3,14317.00,-78.420000,1.647871e+09,16,10,...,-230.378667,0.957805,0.510776,0.000000,32.065938,-292.833333,-29.283333,1.095023,0.477322,2022-03-21
92,92,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 09:05:45.000,SLD,5,14291.75,-295.700000,1.647872e+09,16,11,...,-236.316970,0.933737,0.517133,0.000000,32.065938,-322.403333,-29.309394,1.094050,0.477544,2022-03-21
96,96,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 09:26:21.000,BOT,4,14238.00,324.940000,1.647873e+09,17,11,...,-236.316970,0.959695,0.510284,553.672500,32.568971,0.000000,-29.309394,1.111213,0.473661,2022-03-21
101,101,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-21 09:44:26.000,SLD,5,14270.00,172.800000,1.647874e+09,18,11,...,-236.316970,0.947002,0.513610,570.952500,31.719583,0.000000,-29.309394,1.082233,0.480254,2022-03-21
